In [16]:
from lxml import etree, objectify
import os
import re
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
import pymorphy2 as pm
import pandas as pdr
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [90]:
#функция, которая достает из XML нужные нам данные: год письма и текст письма

def get_data(xml_file):
    with open(xml_file, 'rb') as f:
        xml = f.read()
        root = objectify.fromstring(xml)

        # извлекаем данные.
        year = str(root.teiHeader.fileDesc.titleStmt.year)
        for cld in root.getchildren():
            for elem in cld.getchildren():
                for i in elem.getchildren():
                    text = str(i.text) 
    return year, text

In [ ]:
#функция, которая извлекает текст для w2v

def get_text(xml_file):
    with open(xml_file, 'rb') as f:
        xml = f.read()
        root = objectify.fromstring(xml)

        for cld in root.getchildren():
            for elem in cld.getchildren():
                for i in elem.getchildren():
                    text = str(i.text) 
    return text

In [2]:
os.chdir("D:\МОЯ ПАПКА\УЧЕБА\ВКР\corpus_ver2")

In [3]:
#функциия, которая делит предложения на слова и превращает все в словарь

def tokenize_text(texts):
    texts = [[text for text in t.split()] for t in texts]
    dictionary = corpora.Dictionary(texts)
    
    print(dictionary)
    return dictionary

In [6]:
os.chdir("D:\МОЯ ПАПКА\УЧЕБА\ВКР\corpus_ver2")

In [8]:
#делим все файлы на пять категорий - по годам, создаем списки таких писем

texts_by_year = []
texts41 = []
texts42 = []
texts43 = []
texts44 = []
texts45 = []

for i in os.listdir(): #3565 документов
    year, text = get_data(i)
    if year == '1941':
        texts41.append(text)
    if year == '1942':
        texts42.append(text)
    if year == '1943':
        texts43.append(text)
    if year == '1944':
        texts44.append(text)
    if year == '1945':
        texts45.append(text)
texts_by_year.append(texts41)
texts_by_year.append(texts42)
texts_by_year.append(texts43)
texts_by_year.append(texts44)
texts_by_year.append(texts45)
print(len(texts41))
print(len(texts42))
print(len(texts43))
print(len(texts44))
print(len(texts45))

321
583
1236
896
443


In [167]:
# объединяем все письма в одну строку для каждой категории
for i in texts_by_year:
    i = ' '.join(i)
    string_list.append(i)
print(len(string_list))

5


In [102]:
#разбиваем строки на слова - токены, считаем количество в каждом объединенном тексте
for i in string_list:
    text=re.split(r'\W+', i)
    print(len(text))

46558
133538
258144
161251
82860


In [11]:
# Считаем частотность. Сначала уберем из текстов цифры, знаки препинания и служебные части речи.
# Лемматизируем - приводим оставшиеся слова в начальную форму.
# Эти слова объединяем опять в тексты писем в одну строку - это формат для словаря Gensim.
# Формируем для каждого из текстов словарь, где ключ - слово, а значение - количестве его повторов в тексте.

word_frequency = []
lemmas_list = []
for letters in texts_by_year:
    try:
        lemmas = []
        vocab2 = {}
        for let in letters:
            let = re.sub(r"\d+", "", let)
            let = re.split(r'\W+', let)
            vocab = []
            for i in let:
                try:
                    m=pm.MorphAnalyzer().parse(i)[0]
                    if m not in stopwords.words('russian'):
                        if m.tag.POS not in ['PREP', 'CONJ', 'PRCL', 'INTJ', 'NPRO']:
                            m=m.normal_form
                            vocab.append(m)
                            if m not in vocab2:
                                vocab2[m] = 1
                            else:        
                                vocab2[m] += 1
                except:
                    print('Ошибка анализатора')
            vocab = ' '.join(vocab)
            lemmas.append(vocab)
        print(len(lemmas))
        lemmas_list.append(lemmas)
        word_frequency.append(vocab2)
    except:
        print('Ошибка')
print(len(lemmas_list))

321
583
1236
896
443
5


In [ ]:
string_list = []
for i in lemmas_list:
    i = ' '.join(i)
    string_list.append(i)
    
#стринг лист - список из пяти строк. Каждая строка - лемматизированные тексты всех писем соотвествующего года с сохранением порядка слов в предложении. 
#Используется для Gensim

In [166]:
#на всякий случай сохраним в файл

ith open('strings_for_Gensim.txt', 'a', encoding='utf-8') as file:
    for i in string_list:
        file.write("%s\n" % i)

In [17]:
#смотрим число лемм по годам
for i in word_frequency:
    print(len(i))

3941
7488
9477
7544
4922


In [20]:
#частотность записываем в файл
table.to_csv('letters_word_freq.csv')

In [21]:
# отсортируем слова каждого из текстов по числу повторений
sorted_by_value = []
for i in word_frequency:
    sort = sorted(i.items(), key=lambda kv: kv[1])
    sorted_by_value.append(sort)

In [22]:
#выведем ТОП-20 
for i in sorted_by_value:
    print(f'Текст № {sorted_by_value.index(i)+1}, 20 самых частых слов:')
    print('\n'.join(map(str, i[-20:])))
    print('\n')

Текст № 1, 20 самых частых слов:
('время', 162)
('тот', 167)
('ещё', 168)
('один', 169)
('год', 178)
('наш', 184)
('день', 189)
('ваш', 191)
('живой', 195)
('свой', 214)
('очень', 219)
('получить', 223)
('привет', 236)
('дорогой', 266)
('', 350)
('письмо', 374)
('писать', 381)
('мой', 425)
('весь', 611)
('быть', 612)


Текст № 2, 20 самых частых слов:
('жизнь', 404)
('твой', 406)
('сейчас', 431)
('день', 440)
('дорогой', 445)
('очень', 461)
('привет', 503)
('который', 529)
('ещё', 532)
('год', 544)
('получить', 559)
('один', 578)
('', 602)
('наш', 618)
('свой', 664)
('писать', 915)
('мой', 920)
('письмо', 1119)
('весь', 1582)
('быть', 1943)


Текст № 3, 20 самых частых слов:
('живой', 933)
('ещё', 946)
('который', 949)
('сейчас', 995)
('пока', 1003)
('написать', 1025)
('год', 1047)
('один', 1049)
('дорогой', 1225)
('', 1294)
('получить', 1337)
('наш', 1358)
('свой', 1374)
('мама', 1379)
('привет', 1409)
('мой', 1818)
('писать', 2278)
('письмо', 2935)
('весь', 3013)
('быть', 3074)


Тек

In [83]:
d = {}
for i in sorted_by_value:
    for tpl in i[:20]:
        d[tpl[0]] = tpl[1]
print(d)
        

{'касаточка': 1, 'сознавать': 1, 'слёзка': 1, 'покатиться': 1, 'повидать': 1, 'сахара': 1, 'лесенко': 1, 'нанести': 1, 'варечка': 1, 'внизу': 1, 'стена': 1, 'гнутов': 1, 'сухоборский': 1, 'сибирский': 1, 'полати': 1, 'читаемый': 1, 'козёл': 1, 'варить': 1, 'незавидный': 1, 'комиссия': 1, 'вовик': 1, 'волнение': 1, 'боднары': 1, 'прорыв': 1, 'кузнецов': 1, 'миронов': 1, 'музочка': 1, 'показатель': 1, 'муза': 1, 'клаш': 1, 'черкнуть': 1, 'взамен': 1, 'платье': 1, 'курточка': 1, 'кепка': 1, 'майка': 1, 'вовсе': 1, 'лизончик': 1, 'лизочек': 1, 'кабинет': 1, 'феликс': 1, 'алейников': 1, 'зарваться': 1, 'побратал': 1, 'корош': 1, 'обрушить': 1, 'харьковский': 1, 'орловский': 1, 'парковый': 1, 'заиметь': 1, 'заходз': 1, 'чапаевец': 1, 'черниченко': 1, 'елисеев': 1, 'щербако': 1, 'вым': 1, 'балкуновое': 1, 'яшагин': 1, 'щербинина': 1, 'ряиновый': 1, 'ад': 1, 'внешне': 1, 'звёздочка': 1, 'расхлябать': 1, 'фимчатов': 1, 'лоскутов': 1, 'локов': 1, 'русанов': 1, 'гамолин': 1, 'хозяюшка': 1, 'лёнеч

In [84]:
#для каждой категории выбираем глаголы, рилагательные и существительные
verbs=[]
adjectives=[]
nouns=[]
for i in sorted_by_value:
    v = {}
    a = {}
    n = {}
    for word in i:
        f=pm.MorphAnalyzer().parse(word[0])[0]
        if f.tag.POS=='INFN' or f.tag.POS=='VERB':
            v[word[0]] = word[1]
        if f.tag.POS=='ADJF' or f.tag.POS=='ADJS':
            a[word[0]] = word[1]
        if f.tag.POS=='NOUN':
            n[word[0]] = word[1]  
    print('v', len(v))
    print('a', len(a))
    print('n', len(n))
    verbs.append(v)
    adjectives.append(a)
    nouns.append(n)

v 1051
a 606
n 1909
v 2178
a 1153
n 3488
v 2630
a 1489
n 4557
v 2147
a 1146
n 3590
v 1368
a 737
n 2372


In [45]:
#сохраним эти списки в файлы

table = pd.DataFrame(verbs)
table = table.T
table.to_csv('letters_verbs.csv')

In [46]:
table = pd.DataFrame(adjectives)
table = table.T
table.to_csv('letters_adj.csv')

In [47]:
table = pd.DataFrame(nouns)
table = table.T
table.to_csv('letters_nouns.csv')

# Частотностные таблицы лежат в google-папке
https://drive.google.com/drive/folders/1rV-7-3O8IX-IvG0R53yvtfrJpBAQqXJo
